In [1]:
from sklearn.model_selection import KFold, train_test_split
import torch.distributions.distribution as D
import data.medain_filtering_class as mf
from torch.utils.data import DataLoader
from torch.autograd import Variable
from new_RBM import new_RBM as RBM
import matplotlib.pyplot as plt
import data.read_samples as rs
import torch.optim as optim
import numpy as np
import datetime
import torch

print(datetime.datetime.now(), "model.py code start")

BATCH_SIZE = 10
EPOCH = 100
LEARNING_RATE = 0.2
ANNEALING_RATE = 0.999
VISIBLE_UNITS = [180, 200, 250]
HIDDEN_UNITS = [80, 100, 120]
K_FOLD = 1

[INFO] Pre-processing for make clean
2022-05-13 14:04:07.917878 model.py code start


In [2]:
print("[MODL] Model main code is starting....")

print("[INFO] Read train data, cross-vaildation data and test data from median filtering code")
db1_sig, db1_label, db2_sig, db2_label, db3_sig, db3_label = rs.return_list()

[MODL] Model main code is starting....
[INFO] Read train data, cross-vaildation data and test data from median filtering code
[INFO] Read records file from  ./data/db1/
[RSLT]			 Export records ...
		 ['100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '111', '112', '113', '114', '115', '116', '117', '118', '119', '121', '122', '123', '124']
[INFO]./rdsamp commending start
[IWIP]		rdsamp Converting 100
[IWIP]		rdsamp Converting 101
[IWIP]		rdsamp Converting 102
[IWIP]		rdsamp Converting 103
[IWIP]		rdsamp Converting 104
[IWIP]		rdsamp Converting 105
[IWIP]		rdsamp Converting 106
[IWIP]		rdsamp Converting 107
[IWIP]		rdsamp Converting 108
[IWIP]		rdsamp Converting 109
[IWIP]		rdsamp Converting 111
[IWIP]		rdsamp Converting 112
[IWIP]		rdsamp Converting 113
[IWIP]		rdsamp Converting 114
[IWIP]		rdsamp Converting 115
[IWIP]		rdsamp Converting 116
[IWIP]		rdsamp Converting 117
[IWIP]		rdsamp Converting 118
[IWIP]		rdsamp Converting 119
[IWIP]		rdsamp Converting 121
[IWIP

In [3]:
train_dataloader = DataLoader(db1_sig + db2_sig,
                              batch_size=BATCH_SIZE,
                              num_workers=0, 
                              collate_fn=lambda x: x,
                              shuffle=True)
                              
test_dataloader = DataLoader(db2_sig + db3_sig,
                             batch_size=BATCH_SIZE,
                             num_workers=0, 
                             collate_fn=lambda x: x,
                             shuffle=True)

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
print(device)

cuda


In [5]:
rbm_first = RBM(n_vis=VISIBLE_UNITS[0], n_hid=HIDDEN_UNITS[0], k=K_FOLD, batch=BATCH_SIZE).to(device=device)
rbm_second = RBM(n_vis=VISIBLE_UNITS[1], n_hid=HIDDEN_UNITS[1], k=K_FOLD, batch=BATCH_SIZE).to(device=device)
rbm_third = RBM(n_vis=VISIBLE_UNITS[2], n_hid=HIDDEN_UNITS[2], k=K_FOLD, batch=BATCH_SIZE).to(device=device)

if torch.cuda.is_available():
    first_train_op = optim.Adagrad(rbm_first.parameters(), 0.1)
    second_train_op = optim.Adagrad(rbm_second.parameters(), 0.1)
    third_train_op = optim.Adagrad(rbm_third.parameters(), 0.1)

    gb_first_train_op = optim.Adagrad(rbm_first.parameters(), 0.1)
    gb_second_train_op = optim.Adagrad(rbm_second.parameters(), 0.1)
    gb_third_train_op = optim.Adagrad(rbm_third.parameters(), 0.1)

else:
    first_train_op = optim.SGD(rbm_first.parameters(), 0.1)
    second_train_op = optim.SGD(rbm_second.parameters(), 0.1)
    third_train_op = optim.SGD(rbm_third.parameters(), 0.1)

    gb_first_train_op = optim.SGD(rbm_first.parameters(), 0.1)
    gb_second_train_op = optim.SGD(rbm_second.parameters(), 0.1)
    gb_third_train_op = optim.SGD(rbm_third.parameters(), 0.1)

output_from_first = list()
output_from_second = list()
output_from_third = list()

In [6]:
'''Train Part'''

loss_ = []
for epoch in range(EPOCH):
    '''First bbrbm'''
    for i, (data) in enumerate(train_dataloader):
        if i == 4:
            continue
        data = Variable(
                torch.tensor(data, dtype=torch.float32, device=device)
        ).uniform_(0, 1)
        
        sample_data = torch.bernoulli(data).view(-1, 10)

        # tensor binary
        vog_first, v1, mt = rbm_first(sample_data)
        
        loss_first = rbm_first.free_energy(vog_first) - rbm_first.free_energy(v1)
        loss_.append(loss_first.data)
        
        first_train_op.zero_grad()
        loss_first.backward()
        first_train_op.step()
    
    output_from_first.append(v1.tolist())
    print("1ST BBrbm_first Training loss for {0} epoch {1}\tEstimate time : {2}".format(epoch, np.mean(loss_), mt))


C:\Users\HILAB_~1\AppData\Local\Temp/ipykernel_14660/1971335250.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:210.)
  torch.tensor(data, dtype=torch.float32, device=device)
c:\Users\HILAB_Labtop_02\anaconda3\lib\site-packages\torch\nn\functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


2
1
torch.Size([1300000, 10])
torch.Size([10])
torch.Size([10, 1])


RuntimeError: vector + matrix @ vector expected, got 1, 2, 2

In [ ]:
print(len(output_from_first))
print(output_from_first)

In [ ]:
print(output_from_first.size())

In [ ]:
output_from_first = torch.tensor(output_from_first)
print(output_from_first)

for epoch in range(EPOCH):
    '''Secnd bbrbm'''
    for _, (data) in enumerate(output_from_first): 
        if i == 4:
            continue
        data = Variable(
                torch.tensor(data, dtype=torch.float32)
        ).uniform_(0, 1)

        sample_data = torch.bernoulli(data).view(-1, 10)

        # tensor binary
        vog_second, v2, mt = rbm_second(sample_data)
        
        loss_second = rbm_second.free_energy(vog_second) - rbm_second.free_energy(v2)
        loss_.append(loss_second.data)
        
        second_train_op.zero_grad()
        loss_second.backward()
        second_train_op.step()

    # output_from_second.append(v2)
    print("2ST BBrbm_first Training loss for {0} epoch {1}\tEstimate time : ".format(epoch, np.mean(loss_), mt))


In [ ]:
output_from_second = torch.tensor(output_from_second)
for epoch in range(EPOCH):
    '''Secnd bbrbm'''
    for _, (data) in enumerate(output_from_second):
        if i == 4:
            continue
        data = Variable(
                torch.tensor(data, dtype=torch.float32)
        ).uniform_(0, 1)

        sample_data = torch.bernoulli(data)
        sample_data = torch.flatten(sample_data.clone())

        vog_third, v3, mt = rbm_third(sample_data)
        
        loss_third = rbm_third.free_energy(vog_third) - rbm_third.free_energy(v3)
        loss_.append(loss_second.data)
        
        third_train_op.zero_grad()
        loss_third.backward()
        third_train_op.step()

    output_from_third.append(v3)
    print("3ST BBrbm_first Training loss for {0} epoch {1}\tEstimate time : ".format(epoch, np.mean(loss_), mt))
